In [2]:
from _dfguru import DataFrameGuru as DFG
from _occupancy_forecasting import MasterTrainer
from _occupancy_forecasting import load_data
from _occupancy_forecasting import avoid_name_conflicts
from _evaluating import ParameterSearch

import torch
from torch import nn
from torch.utils.tensorboard import SummaryWriter

import torch.multiprocessing
torch.multiprocessing.set_sharing_strategy('file_system')

import pandas as pd
import numpy as np
import os
dfg = DFG()
torch.cuda.empty_cache()


############ Inputs ############
#args = parse_arguments()
#args = prompt_for_missing_arguments(args)0
#n_run = args.n_run
#n_param = args.n_param

n_run = 10
n_param = 0

overwrite = True
################################

param_dir = "_occupancy_forecasting/parameters/wrap_up"
tb_log_dir = "_occupancy_forecasting/training_logs/wrap_up"
cp_log_dir = "_occupancy_forecasting/checkpoints/wrap_up"
path_to_data = "data/occupancy_forecasting"

frequency = "5min"
split_by = "time"


train_dict, val_dict, test_dict = load_data(
    path_to_data_dir=path_to_data, 
    frequency=frequency, 
    split_by=split_by,
    dfguru=dfg,
    with_examweek=False
)


data = train_dict[0]


In [21]:
# weather data
weather = dfg.load_dataframe("data/weather", "measurements_10min", dtypes=False)

# most important columns: tl_max, tl_min, p_mittel, vv_mittel, ff_max, rfb_mittel, rr, so_h
weather_columns = ["time", "tl", "p", "ff", "ffx", "rf", "rr", "so"]
weather = weather[weather_columns]

weather["time"] = pd.to_datetime(weather["time"], format="%Y-%m-%dT%H:%M+00:00")

# set time as index

weather["ff"] = weather["ff"].fillna(method="ffill")
weather["ffx"] = weather["ffx"].fillna(method="ffill")


In [ ]:
# reasample to 1 min
wather_reindexed = weather.set_index("time")
weather_resampled = wather_reindexed.resample("1min").ffill()

weather_resampled['rr'] /= 10
weather_resampled['so'] /= 10
# round to 2 decimal places
weather_resampled = weather_resampled.round(4)


# rename:
weather_resampled = weather_resampled.rename(columns={
    "tl": "air_temperature",
    "p": "air_pressure",
    "ff": "wind_speed",
    "ffx": "wind_speed_max",
    "rf": "rel_humidity",
    "rr": "precipitation",
    "so": "sunshine_duration"
}).reset_index()


In [28]:
# save to csv
dfg.save_to_csv(weather_resampled, 
                   "", 
                   "weather_for_thesis")

True